In [1]:
# Import statements
import numpy as np
import plotly.graph_objs as go
import pandas as pd
from dotenv import dotenv_values
import plotly.graph_objects as go
SECRETS = dotenv_values("../envs/.env")


In [2]:
TRAIN_DATA_PATH = SECRETS['TRAIN_DATA_PATH']
TEST_DATA_PATH = SECRETS['TEST_DATA_PATH']
PULL_UP_STRENGTH, PULL_DOWN_STRENGTH = list(map(int, SECRETS['BETA'].split(':')))
NUM_SAMPLES = int(SECRETS['NUM_SAMPLES'])

In [4]:
df_inputs = pd.read_csv(TRAIN_DATA_PATH+"\\3nand_pre_processed_data.csv")
df_delays = pd.read_excel(TRAIN_DATA_PATH+"\\graph_results1.xlsx")

def convert_to_picoseconds(delay):
    if delay.endswith('p'):
        # Convert picoseconds to nanoseconds (1 picosecond = 1e-3 nanoseconds)
        return float(delay[:-1]) 
    elif delay.endswith('n'):
        # Nanoseconds remain unchanged
        return float(delay[:-1]) * 1000
    else:
        raise ValueError("Unknown time unit")
    

df_delays['delay'] = df_delays['delay'].apply(convert_to_picoseconds)

# df_delays['delay'] = df_delays['delay'].str.replace('p', '').astype(int) / 1000


In [17]:
k0, k1, k2, delays = list(df_inputs['k0']), list(df_inputs['k1']), list(df_inputs['k2']), list(df_delays['delay'])

In [20]:

fig = go.Figure(data=[go.Scatter3d(
    x=k0,
    y=k1,
    z=k2,
    mode='markers',
    marker=dict(
        size=12,
        color=delays,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(  scene = dict(
                    xaxis_title='k0',
                    yaxis_title='k1',
                    zaxis_title='k2'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

In [19]:
np.corrcoef(delays, k0), np.corrcoef(delays, k1), np.corrcoef(delays, k2)

(array([[ 1.        , -0.12602191],
        [-0.12602191,  1.        ]]),
 array([[ 1.        , -0.41948914],
        [-0.41948914,  1.        ]]),
 array([[1.        , 0.30640226],
        [0.30640226, 1.        ]]))

In [24]:
np.corrcoef(delays, np.array(k2)/np.array(k0))

array([[1.        , 0.23713497],
       [0.23713497, 1.        ]])

**Analysis of the results**:

We have a negative correlation at (delay, k0) and (delay, k1), but there is a positive correlation between (delay, k2)
This result is acceptable and is in line with the theory.

Driver circuit is sized large to get max current through it, higher current, so lesser delay. 

Gate has higher size implies it gives larger current at high sizes and thus lesser delay, thus negative correlation.

load driver acts just like a capacitor with respect to the gate and we measure the delay till before the load driver, so it effectively acts equivalent to a capacitor. Therefore, higher capacitance implies higher delay. Thus, positive correlation.
